In [3]:
# Testing deconvolutions for temp test folder (containing 1 image)

from nrrd import read, write
from tifffile import imwrite
import shutil, os
import time

# delete old directory
try:
    shutil.rmtree(r"E:\TMP\Keivan\de\temp")
    os.remove(r"E:\TMP\Keivan\de\y00010920.x00005472.nrrd")
except FileNotFoundError:
    print("File not found")
    
print("Old directory deleted")
time.sleep(2)

# run fnt_cube_processor.py --no-destripe --wavelength_ex 642 --wavelength_em 690    <- TODO: program crashes
! python .\fnt_cube_processor.py -i E:\TMP\Keivan\or -o E:\TMP\Keivan\de -n 1 -d --no-destripe --nimm 1.42 --dxy 0.7 --dz 1.2
# time.sleep(2)
# img, header = read(r"E:\TMP\Keivan\de\y00010920.x00005472.nrrd")
# imwrite(r"E:\TMP\Keivan\de\test_gd_de_1.42_600.tif", img, compression=1)

# img, header = read(r"E:\TMP\Keivan\or\y00010920.x00005472.nrrd")
# imwrite(r"E:\TMP\Keivan\de\test_or.tif", img, compression=1)

Two objects are distinguishable in xy-plane if they are 801 nm apart. The camera pixel size is 700 nm.
Two objects are distinguishable in z-axis if they are 8662 nm apart. The z-step is 1200 nm.
full width half maxima of xy-plane is 958.9 nm.
full width half maxima of z-axis is 9202.4 nm.
otf shape:  (0, 9, 18)
callednx=17, ny=9, nz=9
Center of mass is (8.000, 4.000, 4.600)
Background is -0.000
In radialft()


C:\Users\ajan\AppData\Local\anaconda3\envs\stitching\Lib\site-packages\pycudadecon\_libwrap.py:44: UserWarning: Unable to find function: camcor_interface_init in libcudaDecon
  @lib.function
C:\Users\ajan\AppData\Local\anaconda3\envs\stitching\Lib\site-packages\pycudadecon\_libwrap.py:51: UserWarning: Unable to find function: camcor_interface in libcudaDecon
  @lib.function

FNT Cube Processor: 0 cubes [00:00, ? cubes/s]
FNT Cube Processor: 0 cubes [00:00, ? cubes/s]


In [2]:
# Testing deconvolutions for temp test folder (containing 1 image)

from nrrd import read, write
from tifffile import imwrite
import shutil, os
import time

# delete old directory
try:
    shutil.rmtree(r"E:\TMP\Aidan\out_test_single\temp")
    os.remove(r"E:\TMP\Aidan\out_test_single\y00006400.x00019200.nrrd")
except FileNotFoundError:
    print("File not found")
    
print("Old directory deleted")
time.sleep(0.5)

# run fnt_cube_processor.py
! python .\fnt_cube_processor.py -i E:\TMP\Aidan\tmp -o E:\TMP\Aidan\out_test_single -n 1 -d  
# time.sleep(2)
img, header = read(r"E:\TMP\Aidan\out_test_single\y00006400.x00019200.nrrd")
imwrite(r"E:\TMP\Aidan\out_test_single\test_decon.tif", img, compression=1)

Old directory deleted
Two objects are distinguishable in xy-plane if they are 801 nm apart. The camera pixel size is 700 nm.
Two objects are distinguishable in z-axis if they are 8662 nm apart. The z-step is 1400 nm.
full width half maxima of xy-plane is 958.9 nm.
full width half maxima of z-axis is 9202.4 nm.
otf shape:  (0, 8, 18)


C:\Users\ajan\AppData\Local\anaconda3\envs\stitching\Lib\site-packages\pycudadecon\_libwrap.py:44: UserWarning: Unable to find function: camcor_interface_init in libcudaDecon
  @lib.function
C:\Users\ajan\AppData\Local\anaconda3\envs\stitching\Lib\site-packages\pycudadecon\_libwrap.py:51: UserWarning: Unable to find function: camcor_interface in libcudaDecon
  @lib.function

FNT Cube Processor: 100%|##########| 1/1 [00:07<00:00,  7.06s/ cubes]


In [ ]:
def convert_file(input_nrrd):
    img, header = read(input_nrrd.__str__())
    tif_path = Path(output_tif_folder) / (input_nrrd.name[:-4] + 'tif')
    imwrite(tif_path.__str__(), img, compression=1)
    return 1

In [ ]:
# Testing deconvolutions for real folder (this one takes a long time)
from nrrd import read
from tifffile import imwrite
from pathlib import Path
from os import system 
from multiprocessing import Pool


# running fnt_cube_processor with 8 threads
input_folder = r"E:\TMP\Aidan\z00003200"
output_nrrd_folder = r"E:\TMP\Aidan\z00003200_deconv_nrrd"
output_tif_folder = r"E:\TMP\Aidan\z00003200_tif"
num_threads = 8

system(fr"python .\fnt_cube_processor.py -i {input_folder} -o {output_nrrd_folder} -n {num_threads} -d")

# convert to tif files
nrrd_paths = [f for f in Path(output_nrrd_folder).iterdir() if f.is_file()]

# convert single file to tif
def convert_file(input_nrrd: Path):
    img, header = read(input_nrrd.__str__())
    tif_path = Path(output_tif_folder) / (input_nrrd.name[:-4] + 'tif')
    imwrite(tif_path.__str__(), img, compression=1)
    return 1

for path in nrrd_paths:
    convert_file(path)

print("All nrrd -> tif conversions completed successfully")


In [ ]:
import nrrd
from pathlib import Path
from numpy import pad
from pycudadecon import decon, make_otf

def process_cube(
        input_file: Path,   # path to nrrd file
        temp_folder: Path,  # path to a temporary folder (folder must exist before function runs)
        psf: str,           # string representing path to psf file
        pad_amount: int,    # amount to pad to each side of image before applying deconvolution
        n_iters: int = 9,
        dz_data: float = 0.7,
        dx_data: float = 0.7,
        dz_psf: float = 1.8,
        dxy_psf: float = 0.2,
        background: int = 0,
        wavelength_em: int = 525,
        na: float = 0.4,
        nimm: float = 1.42
): 
    # read input file
    img, header = nrrd.read(input_file.__str__())

    # generate otf file from psf file
    otf_file = make_otf(
            psf,
            outpath=(temp_folder / 'otf.tif').__str__(),
            dzpsf=dz_psf,
            dxpsf=dxy_psf,
            wavelength=wavelength_em,
            na=na,
            nimm=nimm,
            fixorigin=0,
        )

    print("Image shape before pad: ", img.shape)

    # pad to ensure edges of image are deconvolved
    img = pad(img, pad_amount, 'reflect')
    
    print("Image shape (after pad), before deconvolution: ", img.shape)

    # run deconvolution    
    img = decon(
        img,
        otf_file,
        fpattern=None,
        n_iters=n_iters,
        dzdata=dz_data,
        dxdata=dx_data,
        dzpsf=dz_psf,
        dxpsf=dxy_psf,
        background=background,
        wavelength=wavelength_em,
        na=na,
        nimm=nimm
    )
    print("Image shape after deconvolution: ", img.shape)
    

if __name__ == '__main__':
    # change pad_amount to different integers.  sometimes the deconvolution causes image to change size.
    # .nrrd file along with psf.tif are provided as sample data.

    process_cube(
        Path("C:/path/to/file/y00006400.x00019200.nrrd"),
        Path("C:/path/to/temp/folder"),
        "C:/path/to/psf.tif",
        pad_amount=1
    )